In [2]:
## Validate game moves
### [Demo of the game:](https://www.youtube.com/watch?v=UIB1epCwv4k)
### Three checks to validate the move:
#### 1. Count to check that there are exactly 10 1's.
#### 2.     If so, check to make sure the change results in a contiguous shape
#### 3. Check that there was only one change between previous and new shape.

import pandas as pd
import networkx as nx

In [ ]:


df = pd.read_csv('data/all-games.tsv', sep='\t')
df['timestamp_gallery'] = pd.to_numeric(df['timestamp_gallery'], errors='coerce')
df.head()

## Add a column to the human data file that contains that shape code for the next move's shape. 
for idx, row in df.iterrows():
    if idx > 10:
        break
    print(row[['shape','next_shape']].to_list())

def decode_shape_binaries_str(encoded_str, bits=10):
    """
    Decodes a single string of space-separated decimal codes into
    a 2D list (shape) of 0/1 bits. Each code becomes one row in the shape.

    :param encoded_str: A single string with space-separated decimal values
                        (e.g., "1016 64 64 64").
    :param bits: The fixed width of the binary representation (default=10).
    :return: A list of lists, where each sub-list is a row of bits (0's and 1's).
    """
    # Split the string by spaces to get each code as a separate token
    codes = encoded_str.split()

    shape = []
    for code in codes:
        # Convert the code (string) to an integer
        number = int(code)

        # Convert to binary, left-padded with zeros to the desired bit length
        binary_str = format(number, 'b').rjust(bits, '0')

        # Convert the binary string into a list of integer bits (0 or 1)
        row_of_bits = [int(bit) for bit in binary_str]
        shape.append(row_of_bits)

    return shape
for idx, row in df.iterrows():
    if idx > 4:
        break
    shape = decode_shape_binaries_str(row['shape'])
    shape_matrix = "\n".join(["".join(map(str, line)) for line in shape]) 
    print(shape_matrix)
    print('----------------------')

    next_shape = decode_shape_binaries_str(row['next_shape'])
    next_shape_matrix = "\n".join(["".join(map(str, line)) for line in next_shape]) 
    print(next_shape_matrix)
    print('=======================')

    
# Define a function to check continuity using DFS. Do not execute the search if the total number of 1s is not 10.
# If the DFS count = 10, the function will return True, indicating that the move is valid.
def is_contiguous(shape):
    rows = len(shape)
    cols = len(shape[0])
    visited = set()

    if sum(sum(row) for row in shape) == 10:  # Check if the total number of 1s is 10
        # Determine the starting point, which is the first 1 in the shape
        for r in range(rows):
            for c in range(cols):
                if shape[r][c] == 1:
                    start = (r, c)
                    break
        def dfs(r, c):
            if (r, c) in visited or not (0 <= r < rows and 0 <= c < cols) or shape[r][c] == 0:
                return 0
            visited.add((r, c))
            count = 1 
            for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]: # Check all 4 directions
                count += dfs(r + dr, c + dc)
            return count
        
        return dfs(start[0], start[1]) == 10
    return False
# Example usage:
shape = [
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
]
print(is_contiguous(shape))
    


['1023', '1023']
['1023', '2 1022']
['2 1022', '4 1020 512']
['4 1020 512', '4 1020 256']
['4 1020 256', '520 1016 512']
['520 1016 512', '528 1016 512']
['528 1016 512', '544 1016 512']
['544 1016 512', '576 1016 512']
['576 1016 512', '576 1008 528']
['576 1008 528', '576 1008 544']
['576 1008 544', '576 1008 288']
1111111111
----------------------
1111111111
1111111111
----------------------
0000000010
1111111110
0000000010
1111111110
----------------------
0000000100
1111111100
1000000000
0000000100
1111111100
1000000000
----------------------
0000000100
1111111100
0100000000
0000000100
1111111100
0100000000
----------------------
1000001000
1111111000
1000000000
False
True
False
False
True
False
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True


In [ ]:
# Define a function to check that there is exactly one change between the previous shape and the new shape.

def is_one_change(shape, next_shape):
    rows_shape = len(shape)
    rows_next_shape = len(next_shape)
    cols = len(shape[0])
    
    # Find the row with the greatest overlap in the number of 1s
    max_overlap = 0
    best_offset = 0
    for offset in range(-rows_shape + 1, rows_next_shape):
        overlap = 0
        for r in range(rows_shape):
            if 0 <= r + offset < rows_next_shape:
                overlap += sum(1 for c in range(cols) if shape[r][c] == 1 and next_shape[r + offset][c] == 1)
        if overlap > max_overlap:
            max_overlap = overlap
            best_offset = offset
    
    # Add a row of 0s to the shape with fewer rows
    if rows_shape < rows_next_shape:
        if best_offset > 0:
            shape = [[0] * cols] * best_offset + shape
        else:
            shape = shape + [[0] * cols] * (-best_offset)
    elif rows_shape > rows_next_shape:
        if best_offset > 0:
            next_shape = [[0] * cols] * best_offset + next_shape
        else:
            next_shape = next_shape + [[0] * cols] * (-best_offset)

    change_count = 0
    for r in range(len(shape)):
        for c in range(len(shape[0])):
            if shape[r][c] != next_shape[r][c]:
                change_count += 1
    return change_count == 2  # Exactly one change (one removal and one addition)


# Example usage:
shape = [
    [1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
]
next_shape = [     
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
]
print(is_one_change(shape, next_shape)) 

In [5]:
### Integration of the above functions to check if a move is valid


def is_valid_move(shape, next_shape): # Take in the shape and next_shape as 2D lists
    if is_contiguous(next_shape) and is_one_change(shape, next_shape):
        return True
    return False

# Example usage:
shape = [
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
]

next_shape = [
    [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
]

print(is_valid_move(shape, next_shape)) 


False


In [6]:
# Test the function on the first 20 rows of the dataframe
# Since this data is from humans playing the game, we expect all moves to be valid.


for idx, row in df.iterrows():

    if idx > 20:
        break
    shape = decode_shape_binaries_str(row['shape'])
    next_shape = decode_shape_binaries_str(row['next_shape'])
    print(is_valid_move(shape, next_shape))

# TODO: Address issue with left-justification. Rows 2 and 4 are valid moves, but the function returns False.
# The other False's occur for rows before a gallery save, which is expected because the move validation function is not designed to handle cases where shape = next_shape

False
True
False
True
False
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
True
